## **Notebook Content**

Project: *Customer Prioritization Under Constraints*

File: 05_prioritization_analysis.ipynb <br>
Author: Bryan Melvida

Purpose:
- Translate customer behavioral profiles into execution-relevant treatment constraints
- Assess whether observed behavioral signals justify differentiated customer actions
- Prevent misallocation of effort

Input: [`customer_behavior_profile.parquet`](../data/final/customer_behavior_profile.parquet) <br>

<br>

---

<br>
